In [1]:
## import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
## read data
data = pd.read_csv("combined_transactions.csv")

In [3]:
## filter and keep only purchase
df = data[data['transaction_type'] == 'PurchaseActivity'].reset_index(drop=True)

In [4]:
## get merchant type code is 5732
target_df = df[df['merchant_type_code'] == 5732].reset_index(drop=True)
target_df['datetime'] = pd.to_datetime(target_df['datetime']).dt.date

In [5]:
## group by date and get the average of amount cents
final_df = target_df.groupby('datetime')['amount_cents'].mean().reset_index()
final_df['datetime'] = pd.to_datetime(final_df['datetime'])
final_df = final_df.set_index('datetime')

In [8]:
## shift the amount cents by 10 days and set it as target value we want to predict
final_df = final_df.resample('D', convention='end').asfreq().fillna(method='ffill')
final_df['target'] = final_df['amount_cents'].shift(-10).fillna(method='ffill')


In [10]:
from sklearn.linear_model import Ridge

## use ridge model to predict
clf = Ridge(alpha=10)
clf.fit(final_df['amount_cents'].values.reshape(-1, 1), final_df['target'].values)
final_df['prediction'] = clf.predict(final_df['amount_cents'].values.reshape(-1, 1))
## convert cents into dollars
final_df[['target','prediction']] = final_df[['target','prediction']]/100